# Identificação de Homólogos

In [12]:
import os
from Bio.Blast import NCBIXML
from scripts_python import BLASTer

a = %pwd
wd = (a.rsplit('/',2))[0]
seq_id = "MN908947.3"


In [13]:

"""record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")

result_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query='all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]', )

save_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))
save_file.write(result_handle.read())
save_file.close()
result_handle.close()"""

'record = SeqIO.read(os.path.join( wd,"data/homologue", f"{seq_id}_ORF7a.fasta"), "fasta")\n\nresult_handle = NCBIWWW.qblast("blastn", "nr", record.format("fasta"), short_query=True, expect=0.10, word_size=7, entrez_query=\'all[filter] NOT "Severe acute respiratory syndrome coronavirus 2"[Organism]\', )\n\nsave_file = open(os.path.join( wd,"data/reference", f"{seq_id}_ORF7a_blast.xml", "w"))\nsave_file.write(result_handle.read())\nsave_file.close()\nresult_handle.close()'

## Carregar ficheiro XML da pesquisa

In [14]:
result_handle= open(os.path.join( wd,"data/homologue/ORF7a", "008SHWMV01R-Alignment.xml"))
blast_record_p = NCBIXML.read(result_handle)
result_handle.close()

print('Foram carregadas 100 seq:', len(blast_record_p.alignments))

Foram carregadas 100 seq: 44


## Identificar Homologos

In [1]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

query_len = blast_record_p.query_length

homologue = []

print("Accession", "Identity%")
for aln in blast_record_p.alignments:
    for HSP in aln.hsps:
        if HSP.score > 80 and HSP.expect < 1.0e-10 and HSP.identities/HSP.align_length > 0.80:
            print(aln.accession,HSP.identities/HSP.align_length)
            homologue.append(SeqRecord(Seq(HSP.sbjct), aln.hit_id, "", ""))


print("Total:",len(homologue))

SeqIO.write(homologue, os.path.join(wd,"data/homologue/ORF7a", f"{seq_id}_ORF7a.fasta"), "fasta")

NameError: name 'blast_record_p' is not defined